In [1]:
import pandas as pd
from surprise import Reader, Dataset
from surprise.model_selection import train_test_split
from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBasic, KNNBaseline, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise import accuracy
from surprise.model_selection import cross_validate
import joblib

In [2]:
SVD = joblib.load('./SVD.pkl')

In [3]:
ratings = pd.read_csv("./bgg-19m-reviews.csv", encoding='UTF-8')

In [4]:
gameIds = ratings[ratings['user']=='Torsten']['ID']

In [5]:
print(gameIds)

0            30549
305          68448
655         178900
913          31260
1101        148228
             ...  
18938672     84991
18939019    193134
18942016    136819
18946915    118258
18957980    137596
Name: ID, Length: 1460, dtype: int64


In [6]:
playedgame = gameIds.tolist()
print(playedgame)

[30549, 68448, 178900, 31260, 148228, 84876, 161936, 120677, 98778, 40834, 31481, 36218, 11, 132531, 28143, 822, 167791, 13, 54043, 478, 50, 143884, 6249, 172, 163412, 18602, 42, 204583, 230802, 15062, 183394, 199561, 9220, 148949, 193738, 182028, 2655, 170216, 96848, 2651, 244521, 147020, 146508, 40692, 3955, 34635, 157354, 35677, 70323, 158899, 126163, 266192, 65244, 37111, 14996, 199792, 124361, 3076, 12333, 110327, 124742, 9209, 131357, 121921, 320, 1406, 70919, 181304, 169786, 201808, 162886, 181, 171, 133473, 39856, 72125, 129622, 463, 10547, 173346, 1927, 15987, 107529, 220308, 93, 27162, 432, 176494, 118048, 54, 224037, 263918, 8203, 15512, 198773, 171623, 5782, 43570, 74, 194594, 284083, 12942, 209685, 124708, 258, 244992, 37046, 25613, 233867, 177736, 12, 244522, 25669, 8217, 108745, 104006, 123260, 73439, 147151, 9674, 209778, 21790, 21763, 312484, 122522, 16992, 123540, 92415, 127398, 5, 2453, 100901, 218603, 24480, 153938, 54998, 164928, 116, 118, 18, 77423, 154203, 161970

In [7]:
games = pd.read_csv('./games_detailed_info.csv', encoding='UTF-8')

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_2216\2418803412.py:1: DtypeWarning: Columns (29,48) have mixed types. Specify dtype option on import or set low_memory=False.
  games = pd.read_csv('./games_detailed_info.csv', encoding='UTF-8')


In [9]:
games = pd.read_csv('./games_detailed_info.csv', encoding='UTF-8', usecols=[2,5])
print(games)

           id               primary
0       30549              Pandemic
1         822           Carcassonne
2          13                 Catan
3       68448             7 Wonders
4       36218              Dominion
...       ...                   ...
21626  296892          Sacred Rites
21627  217378                 BABEL
21628   18063              Goldrush
21629   10052           Bongo Kongo
21630  165946  Green Beret: Vietnam

[21631 rows x 2 columns]


In [17]:
def get_unplayed_surprise(ratings, games, userID):
    played_games = ratings[ratings['user']==userID]['ID'].tolist()
    print(played_games)
    unplayed_games = [game for game in total_games if game not in played_games]
    print(f'특정{userID} 유저가 플레이한 게임 수: {len(played_games)}\n 추천한 게임 개수: {len(unplayed_games)}\n 전체 게임수:{len(total_games)}')
    return unplayed_games

In [19]:
def recomm_game_by_surprise(algo, userID, unplayed_games, top_n=10):
    predictions = [algo.predict(str(userID), str(gameId)) for gameId in unplayed_games]

    def sortkey_est(pred):
        return pred.est
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]
    top_game_ids = [int(pred.iid) for pred in top_predictions]
    top_game_ratings = [pred.est for pred in top_predictions]
    top_game_titles = games[games.id.isin(top_game_ids)]['primary']
    top_movie_preds = [(ids, ratings, title) for ids, ratings, title in zip(top_game_ids, top_game_ratings, top_game_titles)]
    return top_movie_preds

In [20]:
unplayed_lst = get_unplayed_surprise(ratings, games, 'Torsten')
top_games_preds = recomm_game_by_surprise(SVD, 'Torsten', unplayed_lst, top_n=10)

[30549, 68448, 178900, 31260, 148228, 84876, 161936, 120677, 98778, 40834, 31481, 36218, 11, 132531, 28143, 822, 167791, 13, 54043, 478, 50, 143884, 6249, 172, 163412, 18602, 42, 204583, 230802, 15062, 183394, 199561, 9220, 148949, 193738, 182028, 2655, 170216, 96848, 2651, 244521, 147020, 146508, 40692, 3955, 34635, 157354, 35677, 70323, 158899, 126163, 266192, 65244, 37111, 14996, 199792, 124361, 3076, 12333, 110327, 124742, 9209, 131357, 121921, 320, 1406, 70919, 181304, 169786, 201808, 162886, 181, 171, 133473, 39856, 72125, 129622, 463, 10547, 173346, 1927, 15987, 107529, 220308, 93, 27162, 432, 176494, 118048, 54, 224037, 263918, 8203, 15512, 198773, 171623, 5782, 43570, 74, 194594, 284083, 12942, 209685, 124708, 258, 244992, 37046, 25613, 233867, 177736, 12, 244522, 25669, 8217, 108745, 104006, 123260, 73439, 147151, 9674, 209778, 21790, 21763, 312484, 122522, 16992, 123540, 92415, 127398, 5, 2453, 100901, 218603, 24480, 153938, 54998, 164928, 116, 118, 18, 77423, 154203, 161970

특정Torsten 유저가 플레이한 게임 수: 1460
 추천한 게임 개수: 20172
 전체 게임수:21631


In [21]:
for top_game in top_games_preds:
    print('추천게임이름',top_game[2])
    print('예측평점:',top_game[1])
    print()     

추천게임이름 Gloomhaven
예측평점: 6.955837815813315

추천게임이름 Dead of Winter: A Crossroads Game
예측평점: 6.955837815813315

추천게임이름 The Resistance
예측평점: 6.955837815813315

추천게임이름 Arkham Horror: The Card Game
예측평점: 6.955837815813315

추천게임이름 Root
예측평점: 6.955837815813315

추천게임이름 Santorini
예측평점: 6.955837815813315

추천게임이름 Eldritch Horror
예측평점: 6.955837815813315

추천게임이름 Sheriff of Nottingham
예측평점: 6.955837815813315

추천게임이름 A Game of Thrones: The Board Game (Second Edition)
예측평점: 6.955837815813315

추천게임이름 Caverna: The Cave Farmers
예측평점: 6.955837815813315



In [22]:
total_games = games['id'].tolist()